# Pràctica II - Perceptron amb Scikit-learn

En la sessió anterior hem implementat i entrenat un perceptró des de zero, analitzant pas a pas el seu funcionament intern. Ara, donarem un pas més i treballarem amb la implementació ja disponible a la llibreria scikit-learn, que ens permetrà concentrar-nos en el procés d’aplicació del model a un conjunt de dades real.

Per tal d’il·lustrar-ho, farem servir el problema del Titanic, que ja coneixeu del curs anterior. L’objectiu és predir si un passatger va sobreviure o no al naufragi a partir d’algunes de les seves característiques (edat, sexe, classe del bitllet, etc.).

Tasques a realitzar

1. Carregar i preparar les dades
   - Importar el conjunt de dades del Titanic.
   - Seleccionar les variables més rellevants i realitzar els preprocessaments necessaris (tractament de valors NaNs, codificació de variables categòriques i normalització).
   - Dividir les dades en conjunts d’entrenament i validació.

2. Definir i entrenar el model
   - Utilitzar la classe Perceptron de ``sklearn.linear_model``.
   - Ajustar el model amb les dades d’entrenament.
   - Utilitzar el perceptró implementat la sessió anterior per comparar resultats.

3. Avaluar el rendiment
   - Calcular la *accuracy* sobre les dades de validació.
   - Analitzar la matriu de confusió per entendre millor els encerts i errors del model.
   - Comparar els resultats de ``scikit-learn`` amb la vostra implementació prèvia.

In [167]:
import pandas as pd
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import Perceptron

Podeu trobar les dades del Titanic a [Kaggle](https://www.kaggle.com/competitions/titanic/overview).

In [168]:
df = pd.read_csv("train.csv")

print(df.head)

<bound method NDFrame.head of      PassengerId  Survived  Pclass  \
0              1         0       3   
1              2         1       1   
2              3         1       3   
3              4         1       1   
4              5         0       3   
..           ...       ...     ...   
886          887         0       2   
887          888         1       1   
888          889         0       3   
889          890         1       1   
890          891         0       3   

                                                  Name     Sex   Age  SibSp  \
0                              Braund, Mr. Owen Harris    male  22.0      1   
1    Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                               Heikkinen, Miss. Laina  female  26.0      0   
3         Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                             Allen, Mr. William Henry    male  35.0      0   
..                                     

## Neteja de dades

### Codificació de variables categòriques

Farem servir la codificació *one-hot* per a les variables categòriques, ``pandas`` té una funció que ens ho facilita: ``get_dummies()``.

In [169]:
df = df.drop(["PassengerId", "Name", "Ticket", "Fare", "Cabin"], axis=1)

print(df.head)

<bound method NDFrame.head of      Survived  Pclass     Sex   Age  SibSp  Parch Embarked
0           0       3    male  22.0      1      0        S
1           1       1  female  38.0      1      0        C
2           1       3  female  26.0      0      0        S
3           1       1  female  35.0      1      0        S
4           0       3    male  35.0      0      0        S
..        ...     ...     ...   ...    ...    ...      ...
886         0       2    male  27.0      0      0        S
887         1       1  female  19.0      0      0        S
888         0       3  female   NaN      1      2        S
889         1       1    male  26.0      0      0        C
890         0       3    male  32.0      0      0        Q

[891 rows x 7 columns]>


### Eliminació de les columnes sense informació

Hi ha algunes columnes que no aporten informació rellevant per a la predicció, com ara el nom del passatger o el número del bitllet. Les hem d'eliminar.

In [170]:
df = pd.get_dummies(df)

print(df.head)

<bound method NDFrame.head of      Survived  Pclass   Age  SibSp  Parch  Sex_female  Sex_male  Embarked_C  \
0           0       3  22.0      1      0       False      True       False   
1           1       1  38.0      1      0        True     False        True   
2           1       3  26.0      0      0        True     False       False   
3           1       1  35.0      1      0        True     False       False   
4           0       3  35.0      0      0       False      True       False   
..        ...     ...   ...    ...    ...         ...       ...         ...   
886         0       2  27.0      0      0       False      True       False   
887         1       1  19.0      0      0        True     False       False   
888         0       3   NaN      1      2        True     False       False   
889         1       1  26.0      0      0       False      True        True   
890         0       3  32.0      0      0       False      True       False   

     Embarked_Q  Emba

### Normalitzam les dades

Hem de normalitzar les dades per tal que totes les característiques tinguin la mateixa escala. Això és especialment important per a algorismes com el perceptró, que són sensibles a la magnitud de les característiques.

In [171]:
scaler = MinMaxScaler()
df_normalized = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)

print(df_normalized)

     Survived  Pclass       Age  SibSp     Parch  Sex_female  Sex_male  \
0         0.0     1.0  0.271174  0.125  0.000000         0.0       1.0   
1         1.0     0.0  0.472229  0.125  0.000000         1.0       0.0   
2         1.0     1.0  0.321438  0.000  0.000000         1.0       0.0   
3         1.0     0.0  0.434531  0.125  0.000000         1.0       0.0   
4         0.0     1.0  0.434531  0.000  0.000000         0.0       1.0   
..        ...     ...       ...    ...       ...         ...       ...   
886       0.0     0.5  0.334004  0.000  0.000000         0.0       1.0   
887       1.0     0.0  0.233476  0.000  0.000000         1.0       0.0   
888       0.0     1.0       NaN  0.125  0.333333         1.0       0.0   
889       1.0     0.0  0.321438  0.000  0.000000         0.0       1.0   
890       0.0     1.0  0.396833  0.000  0.000000         0.0       1.0   

     Embarked_C  Embarked_Q  Embarked_S  
0           0.0         0.0         1.0  
1           1.0         0.0

### Tractament de valors NaN

Els valors NaN els hem de tractar abans d'entrenar el model. Podem optar per eliminar les files amb valors NaN o bé imputar-los amb la mitjana, mediana o moda de la columna. Pandas ens ofereix diverses funcions per fer-ho. Per exemple, podem utilitzar ``fillna()`` per imputar valors, o ``dropna()`` per eliminar files amb valors NaN.

In [172]:
df_normalized["Age"] = df_normalized["Age"].fillna(df_normalized["Age"].median())

print(df_normalized.head)

<bound method NDFrame.head of      Survived  Pclass       Age  SibSp     Parch  Sex_female  Sex_male  \
0         0.0     1.0  0.271174  0.125  0.000000         0.0       1.0   
1         1.0     0.0  0.472229  0.125  0.000000         1.0       0.0   
2         1.0     1.0  0.321438  0.000  0.000000         1.0       0.0   
3         1.0     0.0  0.434531  0.125  0.000000         1.0       0.0   
4         0.0     1.0  0.434531  0.000  0.000000         0.0       1.0   
..        ...     ...       ...    ...       ...         ...       ...   
886       0.0     0.5  0.334004  0.000  0.000000         0.0       1.0   
887       1.0     0.0  0.233476  0.000  0.000000         1.0       0.0   
888       0.0     1.0  0.346569  0.125  0.333333         1.0       0.0   
889       1.0     0.0  0.321438  0.000  0.000000         0.0       1.0   
890       0.0     1.0  0.396833  0.000  0.000000         0.0       1.0   

     Embarked_C  Embarked_Q  Embarked_S  
0           0.0         0.0         1.0

### Divisió del conjunt de dades

Separarem les dades en un conjunt d'entrenament i un de validació. Això ens permetrà avaluar el rendiment del model en dades que no ha vist durant l'entrenament. S'utilitza la funció ``train_test_split()`` de ``sklearn.model_selection``.

In [173]:
x = df_normalized.drop(columns=["Survived"], axis=1)
y = df_normalized["Survived"]

x_train, x_val, y_train, y_val = train_test_split(
    x, y, test_size=0.25, random_state=42
)

print("X_train:")
print(x_train)
print("\nX_val:")
print(x_val)

X_train:
     Pclass       Age  SibSp     Parch  Sex_female  Sex_male  Embarked_C  \
298     0.0  0.346569  0.000  0.000000         0.0       1.0         0.0   
884     1.0  0.308872  0.000  0.000000         0.0       1.0         0.0   
247     0.5  0.296306  0.000  0.333333         1.0       0.0         0.0   
478     1.0  0.271174  0.000  0.000000         0.0       1.0         0.0   
305     0.0  0.006283  0.125  0.333333         0.0       1.0         0.0   
..      ...       ...    ...       ...         ...       ...         ...   
106     1.0  0.258608  0.000  0.000000         1.0       0.0         0.0   
270     0.0  0.346569  0.000  0.000000         0.0       1.0         0.0   
860     1.0  0.509927  0.250  0.000000         0.0       1.0         0.0   
435     0.0  0.170646  0.125  0.333333         1.0       0.0         0.0   
102     0.0  0.258608  0.000  0.166667         0.0       1.0         0.0   

     Embarked_Q  Embarked_S  
298         0.0         1.0  
884         0.0   

## Entrenament i avaluació del model

Hem d'entrenar el perceptró amb les dades d'entrenament i després avaluar-lo amb les dades de validació. Hem de fer dos entrenaments, un amb la nostra implementació i un altre amb la de ``scikit-learn``. Per emprar la de ``scikit-learn``, utilitzarem la classe ``Perceptron`` de ``sklearn.linear_model``.

In [174]:
model = Perceptron(max_iter=1000, random_state=42)
model.fit(x_train, y_train)

y_pred = model.predict(x_val)

In [175]:
acc = accuracy_score(y_val, y_pred)
print("Accuracy:", acc)

cm = confusion_matrix(y_val, y_pred)
print("Matriz de confusion: \n", cm)

Accuracy: 0.7668161434977578
Matriz de confusion: 
 [[102  32]
 [ 20  69]]
